In [ ]:
 # Import libraries necessary for this project
import numpy as np
import pandas as pd
import requests
import csv
import seaborn as sns
import mpl_toolkits
import statsmodels.api as sm
from scipy import stats
from sklearn import preprocessing

In [ ]:
def api_property_list_sold(city, state, prop_type, limit=200):
    
  # url for api
  url = "https://realtor.p.rapidapi.com/properties/v2/list-sold"

  # enter parameters
  querystring = {
    "sort":"sold_date",
    "city":city,
    "offset":"0",
    "state_code":state,
    "limit":limit,
    "prop_type":prop_type
  }

  headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': "95338ea407msh3cf14ebb0eb2e8dp107b98jsnbd45c26c0423"
  }

  response = requests.request("GET", url, headers=headers, params=querystring)
  return response.json() # json format

In [ ]:
def process_list_sold_response(response_json):

    """
    Process the list for sale API response.

    Convert each listing to a dataframe, append to a list, and concatenate to one dataframe.

    Parameters
    ----------
    @response_json [dictionary]: API response for list sold

    Returns
    -------
    [dataframe] Dataframe of all list sold responses

    """

    # empty dataframe
    dataframe_list = []

    # iterate through each sold listing
    for l in response_json['properties']:

        # convert each listing to dataframe
        _temp_df = pd.DataFrame.from_dict(l, orient='index').T

        # append to dataframe list for all listings
        dataframe_list.append(_temp_df)

    # concatenate all dataframes, for missing col values enter null value
    return pd.concat(dataframe_list, axis=0, ignore_index=True, sort=False)

In [ ]:
# api key to access data
city = "Gainesville"
state = "FL"
prop_type = "single_family"

In [ ]:
property_list_sold_response = api_property_list_sold(city=city, 
                                                     state=state, 
                                                     prop_type=prop_type,
                                                     limit=200)

In [ ]:
property_list_sold_response

In [ ]:
df_properties_sold_raw = process_list_sold_response(response_json=property_list_sold_response)
df_properties_sold_raw.head()

In [ ]:
# Output dataframe to CSV so that project data will stay consistent
df_properties_sold_raw.to_csv(r'C:\Users\green\OneDrive\Documents\DSC680\Project1\HomePricesGainesville071320.csv', 
                              index=False, sep=',',encoding='utf-8')

In [ ]:
df = pd.read_csv(r'C:\Users\green\OneDrive\Documents\DSC680\Project1\HomePricesGainesville071320.csv')
df.head()

In [ ]:
# Drop columns that are identifiers only
df.drop(columns=['property_id', 'listing_id', 'prop_type', 'list_date', 'last_update', 'prop_status', 'address', 'mls', 
                 'client_display_flags', 'sold_history', 'office', 'agents', 'rdc_web_url', 'rdc_app_url', 'data_source_name', 
                 'page_no', 'rank', 'list_tracking', 'photos', 'price_reduced_date'], inplace=True)
df.head()

In [ ]:
# Replace 'NaN' values with '0'
df= df.replace(np.nan,0)
df.head()

In [ ]:
def trim(dftotrim):
    # Trim off size and unit strings 
           
    dftotrim['lot_size'] = dftotrim['lot_size'].str.extract('(\d+)')
    dftotrim['building_size'] = dftotrim['building_size'].str.extract('(\d+)').astype(int)

In [ ]:
trim(df)
# Output dataframe to CSV so that project data can be reviewed
df.to_csv(r'C:\Users\green\OneDrive\Documents\DSC680\Project1\HomePricesGainesville071320trim.csv', 
                              index=False, sep=',',encoding='utf-8')
df.head()

In [ ]:
#  Univariate Statistics

def unistats(df):
    # Create some statistics
    
    output_df = pd.DataFrame(columns=['Count', 'Missing', 'Unique', 'Dtype', 'Mean', 'Mode', 'Min', '25%', 'Median', '75%', 'Max', 'Std', 'Skew', 'Kurt'])
    
    for col in df:
        if pd.api.types.is_numeric_dtype(df[col]):
            output_df.loc[col] = [df[col].count(), df[col].isnull().sum(), df[col].nunique(), df[col].dtype, df[col].mean(), 
                                  df[col].mode().values[0], df[col].min(), df[col].quantile(.25), df[col].median(), 
                                  df[col].quantile(.75), df[col].max(), df[col].std(), df[col].skew(), df[col].kurt()]
        else:
            output_df.loc[col] = [df[col].count(), df[col].isnull().sum(), df[col].nunique(), df[col].dtype, '-', 
                                  df[col].mode().values[0], '-', '-', '-', '-', '-', '-', '-', '-']
    return output_df

In [ ]:
unistats(df)

In [ ]:
# There are 3 entries where lot size is missing.  Remove those.
df['lot_size'].replace('', np.nan, inplace=True)
df.dropna(subset=['lot_size'], inplace=True)
unistats(df)


In [ ]:
# Change variable type from float or object to int
df['garage'] = df['garage'].astype(int)
df['baths_half'] = df['baths_half'].astype(int)
df['lot_size'] = df['lot_size'].astype(int)
unistats(df)

In [ ]:
# Bivariate Statistics

In [ ]:
def bivstats(df, label):
    #Create empty DataFrame to store output
    output_df = pd.DataFrame(columns = ['r', 'p-value'])
    # r = Pearson Correlation
    
    for col in df: 
        if not col == label:      
            r, p = stats.pearsonr(df[label], df[col])
            output_df.loc[col] = [round(r, 3), round(p,5)]
    return output_df.sort_values(by=['r'], ascending=False)

In [ ]:
bivstats(df, 'price')

In [ ]:
# Remove new construction feature
df.drop(columns=['is_new_construction'], inplace=True)

In [ ]:
# scatter plots with histograms
def scatter(df, label):
    sns.set(color_codes = True)
    for col in df:        
        if not col == label:
            sns.jointplot(df[col], df[label], kind = 'reg')

In [ ]:
scatter(df, 'price')

In [ ]:
# Investigate outliers 
# lot_size has 2 points that should be removed
# We can remove any lot_size entries above 200000
df.drop(df[df.lot_size > 200000].index, inplace=True)

In [ ]:
#get new stats
bivstats(df, 'price')

In [ ]:
# Draw new scatter plots
scatter(df, 'price')

In [ ]:
# Multivariate Statistics and Model Automation

In [ ]:
# Perform scaling and preprocessing
def mlr_prepare(df):
    df = df.select_dtypes(np.number)
    df_minmax = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df), columns = df.columns)
    return df_minmax

df_minmax = mlr_prepare(df)
df_minmax.head()

In [ ]:
# Run MLR (Multivariate Linear Regression)
def mlr(df, label):
    y = df[label]
    X = df.drop(columns = [label]).assign(const = 1)
    results = sm.OLS(y, X).fit()
    return results

results = mlr(df, 'price')
results.summary()

In [ ]:
# Calculate fit stats and create a record entry for modeling results table
def mlr_fit(results, actual, roundto=10):
    #generate feature table that allows sorting coef labels based on t and p
    df_features = mlr_feature_df(results)
    residuals = np.array(actual) - np.array(results.fittedvalues)
    rmse = np.sqrt(sum((residuals**2))/len(actual))
    mae = np.mean(abs(np.array(actual) - np.array(results.fittedvalues)))
    fit_stats = [round(results.rsquared, roundto), round(results.rsquared_adj, roundto),
                round(results.rsquared - results.rsquared_adj, roundto), round(rmse, roundto), 
                round(mae, roundto), [df_features.index.values]]
    return fit_stats

fit_metrics_list = mlr_fit(results, df['price'])
fit_metrics_list

In [ ]:
# Generate DataFrame that allows us to sort features by t and p values
def mlr_feature_df(results):
    df_features = pd.DataFrame({'coef':results.params, 't':abs(results.tvalues), 'p':round(results.pvalues,6)})
    df_features.drop(labels = ['const'], inplace = True)
    df_features = df_features.sort_values(by = ['t', 'p'])
    return df_features

df_features = mlr_feature_df(results)
df_features

In [ ]:
# Control mlr and mlr_fit by removing certain features
def mlr_step(df, label, min = 2):
    #create empty model results table
    df_models = pd.DataFrame(columns = ['R2', 'R2a', 'diff', 'RMSE', 'MAE', 'features'])
    #prepare data by generating dummies and scaling
    df = mlr_prepare(df)
    #run first model with all features
    results = mlr(df, label)
    #generate fit stats for model
    df_models.loc[str(len(results.params))] = mlr_fit(results, df[label], 10)
    #generate feature table that allows sorting coef labels based on t and p
    df_features = mlr_feature_df(results)
    #step through series of reduced models until minimum # of features left
    while len(results.params) >= min:
        df = df.drop(columns = [df_features.index[0]])  #drop least effective feature
        results = mlr(df, label)                        #re-run next MLR
        df_features = mlr_feature_df(results)           #re-generate the features summary table
        df_models.loc[len(results.params)] = mlr_fit(results, df[label], 10)
    #save full models table to a csv
    df_models.to_csv('C:/Users/green/OneDrive/Documents/DSC680/Project1/' + label + '.csv')
    #return a shortened version without feature list
    df_models.drop(columns = ['features'], inplace = True)
    return df_models

In [ ]:
df_models = mlr_step(df, 'price')
df_models

In [ ]:
# Analysis
#
# Looking at R2, it starts to decline more rapidly after the 8rd feature is removed.
# By looking at our price.csv that was created, we can see that we can remove: 
# photo_count (# of photos in the listing)
# baths_full (# of full bathrooms)
# baths_half (# of half bathrooms)
# baths (total # of bathrooms, half or full, in integers)
# year-built (year in which the house was built)
# beds (# of bedrooms)
#
# All of these features, once removed, hardly make a difference in R2, R2a, RMSE, or MAE.
# 
# The remaining features in decreasing importance are:
# building_size (square footage of house)
# garage (size of garage by # of cars it can fit)
#
# This really surprises me that the garage size is the second most important feature!


In [ ]:
# Now lets remove the extraneous features and get a MLR equation
dfnew = df.drop(columns = ['photo_count', 'baths_full', 'baths_half', 'baths', 'year_built', 'beds', 'lot_size'])

results = mlr(dfnew, 'price')
fit_metrics_list = mlr_fit(results, df['price'])
df_features = mlr_feature_df(results)
df_features

In [ ]:
# So our formula to predict price of a house would be price = (32248.8 * garage) + (136.8 * building_size)
# lets test with the 1st entry: price = (32248.8 * 2) + (136.8 * 2715) = 435909.6
# Actual price is 415000, off by 5%.  Lets add 'beds' back and see how we do.

In [ ]:
dfnewer = df.drop(columns = ['photo_count', 'baths_full', 'baths_half', 'baths', 'year_built', 'lot_size'])

results = mlr(dfnewer, 'price')
fit_metrics_list = mlr_fit(results, df['price'])
df_features = mlr_feature_df(results)
df_features

In [ ]:
# So our formula to predict price of a house would be price = (31145.8 * garage) + (151.8 * building_size) - (20869.2 * beds)
# lets test with the 1st entry: price = (31145.8 * 2) + (151.8 * 2715) - (20869.2 * 4) = 390951.8
# Actual price is 415000, off by 6%.  It appears our original assessment might be better.

In [ ]:
# The price of a house in Gainesville, FL can be determined with ~95% accuracy with the following formula:
# price = 137x + 32249y, where x is the building size in sqft, and y is the size of the garage in cars.